In [1]:
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
import pandas as pd
import numpy as np 

import create_model
import pickle
import time 
import copy
import random
import data_formatting

In [2]:
#dataset = 'enron'

#vocab_dict = pickle.load(open('../processed_data/word_dict_v02_enron_py35_seq_length_3_49_sample_4256_limited_vocab.pkl', 'rb'))
#df_all = pd.read_pickle('../processed_data/processed_data_v02_enron_py35_seq_length_3_49_sample_4256_limited_vocab.pkl')

dataset = 'twitter'

#vocab_dict = pickle.load(open('../processed_data/word_dict_v02_twitter_py35_seq_length_3_25_sample_1764604_questions.pkl', 'rb'))
#df_all = pd.read_pickle('../processed_data/processed_data_v02_twitter_py35_seq_length_3_25_sample_1764604_questions.pkl')

#vocab_dict = pickle.load(open('../processed_data/word_dict_v02_twitter_py35_seq_length_4_15_sample_134241_full.pkl', 'rb'))
#df_all = pd.read_pickle('../processed_data/processed_data_v02_twitter_py35_seq_length_4_15_sample_134241_full.pkl')

vocab_dict = pickle.load(open('../processed_data/word_dict_v02_twitter_py35_seq_length_3_19_sample_21946_lem.pkl', 'rb'))
df_all = pd.read_pickle('../processed_data/processed_data_v02_twitter_py35_seq_length_3_19_sample_21946_lem.pkl')

#vocab_dict = pickle.load(open('../processed_data/word_dict_v02_twitter_py35_seq_length_3_25_sample_1901567_full.pkl', 'rb'))
#df_all = pd.read_pickle('../processed_data/processed_data_v02_twitter_py35_seq_length_3_25_sample_1901567_full.pkl')

In [3]:
inv_map = data_formatting.createInvMap(vocab_dict)

In [4]:
df_all['alpha_Pair_0_encoding'] = df_all['alpha_Pair_0_tokens'].apply(lambda x: data_formatting.encodeSent(x, vocab_dict))
df_all['alpha_Pair_1_encoding'] = df_all['alpha_Pair_1_tokens'].apply(lambda x: data_formatting.encodeSent(x, vocab_dict))
df_all['Index'] = df_all.index.values

In [5]:
df_all_train = df_all.sample(frac=0.97, random_state=1)

df_all_dev = df_all[df_all['Index'].isin(df_all_train['Index'].values) == False]

df_all_test = df_all_dev.sample(frac=0.10, random_state=1)

df_all_dev = df_all_dev[df_all_dev['Index'].isin(df_all_test['Index'].values) == False]

In [6]:
print (df_all.shape[0], df_all_train.shape[0],  df_all_dev.shape[0], df_all_test.shape[0], len(vocab_dict))

17690 17159 478 53 17322


In [7]:
train_data = data_formatting.prepare_train_batch(df_all_train['alpha_Pair_0_encoding'].values, 
                                                    df_all_train['alpha_Pair_1_encoding'].values)

dev_data = data_formatting.prepare_train_batch(df_all_dev['alpha_Pair_0_encoding'].values, 
                                                    df_all_dev['alpha_Pair_1_encoding'].values)

test_data = data_formatting.prepare_train_batch(df_all_test['alpha_Pair_0_encoding'].values, 
                                                    df_all_test['alpha_Pair_1_encoding'].values)

In [8]:
train_model_params = {'n_cells':256, 'num_layers':2, 'embedding_size':512, 
          'vocab_size':len(vocab_dict) + 1, 'minibatch_size':64, 'n_threads':128,
          'beam_width':10, 'limit_decode_steps': None,
          'encoder_input_keep':1, 'decoder_input_keep':1,
          'encoder_output_keep':1, 'decoder_output_keep':1,
         }

In [9]:
dev_model_params = train_model_params
dev_model_params['encoder_input_keep'] = 1
dev_model_params['encoder_output_keep'] = 1
dev_model_params['decoder_input_keep'] = 1
dev_model_params['decoder_output_keep'] = 1

In [10]:
training_params = { 'vocab_lower':3, 'vocab_upper':train_model_params['vocab_size']-1, 
                    'n_epochs':500000}

In [822]:
  enc input           > ['i', 'll', 'be', 'there', 'tomorrow', '<EOS>']
    dec input           > ['last', 'game', 'of', 'the', 'regular', 'season', '<EOS>']

{'beam_width': 10,
 'decoder_input_keep': 1,
 'decoder_output_keep': 1,
 'embedding_size': 512,
 'encoder_input_keep': 1,
 'encoder_output_keep': 1,
 'limit_decode_steps': None,
 'minibatch_size': 64,
 'n_cells': 256,
 'n_threads': 128,
 'num_layers': 2,
 'vocab_size': 17323}

In [11]:
from tensorflow.python.framework import ops
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import control_flow_ops
from tensorflow.python.framework import dtypes
from tensorflow.python.ops import nn_ops
from tensorflow.python.framework import tensor_util
from tensorflow.python.framework import tensor_shape
from tensorflow.python.util import nest
from tensorflow.python.ops import math_ops
from tensorflow.python.ops import tensor_array_ops

In [12]:
tf.reset_default_graph()

inf_model_1_params = {'n_cells':256, 'num_layers':2, 'embedding_size':512, 
          'vocab_size':len(vocab_dict) + 1, 'minibatch_size':64, 'n_threads':128,
          'beam_width':1, 'limit_decode_steps': None,
          'encoder_input_keep':1, 'decoder_input_keep':1,
          'encoder_output_keep':1, 'decoder_output_keep':1,
         }

#inf_model_2_params = {'n_cells':2, 'num_layers':1, 'embedding_size':5, 
#            'vocab_size':7,
#          'minibatch_size':64, 'n_threads':128,
#          'beam_width':3, 'limit_decode_steps': None,
#          'encoder_input_keep':1, 'decoder_input_keep':1,
#          'encoder_output_keep':1, 'decoder_output_keep':1,
#         }


inf_model_2_params = {'n_cells':256, 'num_layers':2, 'embedding_size':512, 
          'vocab_size':len(vocab_dict) + 1,
          'minibatch_size':64, 'n_threads':128,
          'beam_width':3, 'limit_decode_steps': None,
          'encoder_input_keep':1, 'decoder_input_keep':1,
          'encoder_output_keep':1, 'decoder_output_keep':1,
         }

#with tf.variable_scope('training_model'):

#    inf_model_1 = create_model.Model(inf_model_1_params, 'infer', None)
    
with tf.variable_scope('training_model'):

    inf_model_2 = create_model.Model(inf_model_2_params, 'infer', None)   

#with tf.variable_scope('training_model'):

#    inf_train = create_model.Model(inf_model_2_params, 'debug', None)   

In [13]:
batch_size = inf_model_2.inference_decoder._batch_size
beam_width = inf_model_2.inference_decoder._beam_width
end_token = inf_model_2.inference_decoder._end_token
length_penalty_weight = inf_model_2.inference_decoder._length_penalty_weight

In [14]:
name= None

In [15]:
finished, first_inputs, initial_state = inf_model_2.inference_decoder.initialize()

In [16]:
time = 1

In [17]:
#When we pass time !=0 to inference decoder, even with initial step results we get non-zero parent ids

beam_search_output, beam_search_state, next_inputs, finished = inf_model_2.inference_decoder.step(time, first_inputs, initial_state)

In [18]:
input_user = data_formatting.encodeSent(['i', 'll', 'be', 'there', 'tomorrow', '<EOS>'], vocab_dict)
with tf.Session() as sess:
    #sess.run(tf.global_variables_initializer())
    #tf.set_random_seed(1)
    saver = tf.train.Saver()
   
    saver.restore(sess, '../chkpt/seq2seq_twitter_testing-5501')
    q_0 = sess.run([first_inputs, initial_state, beam_search_output, beam_search_state, next_inputs, finished], 
                 feed_dict={'training_model/encoder_inputs:0':[input_user], 
                         'training_model/encoder_inputs_length:0':[len(input_user)]})

INFO:tensorflow:Restoring parameters from ../chkpt/seq2seq_twitter_testing-5501


In [19]:
#with ops.name_scope(name, "BeamSearchDecoderStep", (time, first_inputs, initial_state)):
    
cell_state = initial_state.cell_state

first_inputs = nest.map_structure(lambda inp: inf_model_2.inference_decoder._merge_batch_beams(inp, s=inp.shape[2:]), 
                                  first_inputs)

cell_state = nest.map_structure(inf_model_2.inference_decoder._maybe_merge_batch_beams, cell_state, 
                                inf_model_2.inference_decoder._cell.state_size)

##This goes through _cell first
#The output of cell_outputs is also present in the next_cell_state, the cell_outputs is the attention

#The cell_state includes, the cell and hidden states of the LSTM, the attention states, the alignment history, and timestep, 
#what does alignment history and attention states really mean? need to internalize...

cell_outputs, next_cell_state = inf_model_2.inference_decoder._cell(first_inputs, cell_state)

In [20]:
cell_outputs = nest.map_structure(lambda out: inf_model_2.inference_decoder._split_batch_beams(out, out.shape[1:]), 
                                  cell_outputs)

next_cell_state = nest.map_structure(inf_model_2.inference_decoder._maybe_split_batch_beams, next_cell_state, 
                                     inf_model_2.inference_decoder._cell.state_size)

if inf_model_2.inference_decoder._output_layer is not None:
    cell_outputs = inf_model_2.inference_decoder._output_layer(cell_outputs)

In [100]:
input_user = data_formatting.encodeSent(['i', 'll', 'be', 'there', 'tomorrow', '<EOS>'], vocab_dict)
with tf.Session() as sess:
    #sess.run(tf.global_variables_initializer())
    #tf.set_random_seed(1)
    saver = tf.train.Saver()
   
    saver.restore(sess, '../chkpt/seq2seq_twitter_testing-5501')
    q_0 = sess.run([cell_outputs, next_cell_state], 
                 feed_dict={'training_model/encoder_inputs:0':[input_user], 
                         'training_model/encoder_inputs_length:0':[len(input_user)]})

INFO:tensorflow:Restoring parameters from ../chkpt/seq2seq_twitter_testing-5501


In [106]:
q_0[0][0]

array([[-15.48710155,  -8.94941521,  -9.60930252, ...,   1.44874358,
         -7.69063377, -10.08347988],
       [-15.4871006 ,  -8.94941616,  -9.60930252, ...,   1.4487443 ,
         -7.69063282, -10.08348179],
       [-15.4871006 ,  -8.94941616,  -9.60930061, ...,   1.44874334,
         -7.6906333 , -10.08348083]], dtype=float32)

In [23]:
#cell_outputs_rnn_call, next_cell_state_rnn_call = inf_model_2.inference_decoder._cell.call(first_inputs, cell_state)

#cell_outputs_rnn_call_call, next_cell_state_rnn_call_call = inf_model_2.inference_decoder._cell.call(cell_outputs_rnn_call, 
#                                                                                                     next_cell_state_rnn_call)

In [24]:
sess.close()

In [25]:
static_batch_size = tensor_util.constant_value(batch_size)

In [26]:
prediction_lengths = initial_state.lengths
previously_finished = initial_state.finished

In [27]:
end_token = data_formatting.EOS

In [28]:
def _tensor_gather_helper(gather_indices, gather_from, batch_size,
                          range_size, gather_shape):
  """Helper for gathering the right indices from the tensor.
  This works by reshaping gather_from to gather_shape (e.g. [-1]) and then
  gathering from that according to the gather_indices, which are offset by
  the right amounts in order to preserve the batch order.
  Args:
    gather_indices: The tensor indices that we use to gather.
    gather_from: The tensor that we are gathering from.
    batch_size: The input batch size.
    range_size: The number of values in each range. Likely equal to beam_width.
    gather_shape: What we should reshape gather_from to in order to preserve the
      correct values. An example is when gather_from is the attention from an
      AttentionWrapperState with shape [batch_size, beam_width, attention_size].
      There, we want to preserve the attention_size elements, so gather_shape is
      [batch_size * beam_width, -1]. Then, upon reshape, we still have the
      attention_size as desired.
  Returns:
    output: Gathered tensor of shape tf.shape(gather_from)[:1+len(gather_shape)]
  """
  range_ = array_ops.expand_dims(math_ops.range(batch_size) * range_size, 1)
  gather_indices = array_ops.reshape(gather_indices + range_, [-1])
  output = array_ops.gather(
      array_ops.reshape(gather_from, gather_shape), gather_indices)
  final_shape = array_ops.shape(gather_from)[:1 + len(gather_shape)]
  static_batch_size = tensor_util.constant_value(batch_size)
  final_static_shape = (tensor_shape.TensorShape([static_batch_size])
                        .concatenate(
                            gather_from.shape[1:1 + len(gather_shape)]))
  output = array_ops.reshape(output, final_shape)
  output.set_shape(final_static_shape)
  return output

def _mask_probs(probs, eos_token, finished):
  """Masks log probabilities.
  The result is that finished beams allocate all probability mass to eos and
  unfinished beams remain unchanged.
  Args:
    probs: Log probabiltiies of shape `[batch_size, beam_width, vocab_size]`
    eos_token: An int32 id corresponding to the EOS token to allocate
      probability to.
    finished: A boolean tensor of shape `[batch_size, beam_width]` that
      specifies which
      elements in the beam are finished already.
  Returns:
    A tensor of shape `[batch_size, beam_width, vocab_size]`, where unfinished
    beams stay unchanged and finished beams are replaced with a tensor with all
    probability on the EOS token.
  """
  vocab_size = array_ops.shape(probs)[2]
  finished_mask = array_ops.expand_dims(
      math_ops.to_float(1. - math_ops.to_float(finished)), 2)
  # These examples are not finished and we leave them
  non_finished_examples = finished_mask * probs
  # All finished examples are replaced with a vector that has all
  # probability on EOS
  finished_row = array_ops.one_hot(
      eos_token,
      vocab_size,
      dtype=probs.dtype,
      on_value=0.,
      off_value=probs.dtype.min)
  finished_examples = (1. - finished_mask) * finished_row
  return finished_examples + non_finished_examples

In [29]:
logits = cell_outputs
step_log_probs = nn_ops.log_softmax(logits)
step_log_probs = _mask_probs(step_log_probs, end_token, previously_finished)
total_probs = array_ops.expand_dims(initial_state.log_probs, 2) + step_log_probs

In [30]:
vocab_size = logits.shape[-1].value or array_ops.shape(logits)[-1]

In [31]:
lengths_to_add = array_ops.one_hot(
      indices=array_ops.tile(
          array_ops.reshape(end_token, [1, 1]), [batch_size, beam_width]),
      depth=vocab_size,
      on_value=0,
      off_value=1)

In [98]:
input_user = data_formatting.encodeSent(['i', 'll', 'be', 'there', 'tomorrow', '<EOS>'], vocab_dict)
with tf.Session() as sess:
    #sess.run(tf.global_variables_initializer())
    #tf.set_random_seed(1)
    saver = tf.train.Saver()
   
    saver.restore(sess, '../chkpt/seq2seq_twitter_testing-5501')
    q_0 = sess.run([total_probs, logits], 
                 feed_dict={'training_model/encoder_inputs:0':[input_user], 
                         'training_model/encoder_inputs_length:0':[len(input_user)]})

INFO:tensorflow:Restoring parameters from ../chkpt/seq2seq_twitter_testing-5501


In [99]:
q_0[1]

array([[[-15.48709774,  -8.9494133 ,  -9.60930061, ...,   1.44874406,
          -7.69063282, -10.08347797],
        [-15.48709774,  -8.9494133 ,  -9.60930061, ...,   1.44874406,
          -7.69063282, -10.08347797],
        [-15.48709869,  -8.9494133 ,  -9.60930157, ...,   1.44874334,
          -7.69063282, -10.08347893]]], dtype=float32)

In [97]:
sess.close()

In [34]:
add_mask = (1 - math_ops.to_int32(previously_finished))

lengths_to_add = array_ops.expand_dims(add_mask, 2) * lengths_to_add

new_prediction_lengths = (lengths_to_add + array_ops.expand_dims(prediction_lengths, 2))

In [35]:
scores = total_probs

In [36]:
scores_shape = array_ops.shape(scores)

In [37]:
time = ops.convert_to_tensor(time, name="time")

In [38]:
scores_flat = control_flow_ops.cond(time > 0,
      lambda: array_ops.reshape(scores, [batch_size, -1]), lambda: scores[:, 0])

In [39]:
num_available_beam = control_flow_ops.cond(
      time > 0, lambda: math_ops.reduce_prod(scores_shape[1:]),
      lambda: math_ops.reduce_prod(scores_shape[2:]))

In [40]:
# Pick the next beams according to the specified successors function
next_beam_size = math_ops.minimum(
  ops.convert_to_tensor(beam_width, dtype=dtypes.int32, name="beam_width"),
  num_available_beam)

In [41]:
next_beam_scores, word_indices = nn_ops.top_k(scores_flat, k=next_beam_size)
next_beam_scores.set_shape([static_batch_size, beam_width])
word_indices.set_shape([static_batch_size, beam_width])

In [42]:
next_beam_probs = _tensor_gather_helper(
      gather_indices=word_indices,
      gather_from=scores,
      batch_size=batch_size,
      range_size=beam_width * vocab_size,
      gather_shape=[-1])

In [43]:
#Here, word indices represents the positions of the flattened list of beams, which goes from 1 to beam_width*vocab_size
#thus the word indices obtained are not the word indices of the 1 to N vocab but the flattened out list of logprobs from 1 to 
#beam_width*vocab_size
next_word_ids = math_ops.to_int32(word_indices % vocab_size)
#The beam_ids represent which beams these word indices (and thereby log prob values belong to), as
#we have word indices extracted out of 1 to beam_width*vocab_size
next_beam_ids = math_ops.to_int32(word_indices / vocab_size)

In [75]:
input_user = data_formatting.encodeSent(['i', 'll', 'be', 'there', 'tomorrow', '<EOS>'], vocab_dict)
with tf.Session() as sess:
    #sess.run(tf.global_variables_initializer())
    #tf.set_random_seed(1)
    saver = tf.train.Saver()
   
    saver.restore(sess, '../chkpt/seq2seq_twitter_testing-5501')
    q_0 = sess.run([next_word_ids, next_beam_ids, scores, scores_flat, next_beam_scores, word_indices, next_beam_size], 
                 feed_dict={'training_model/encoder_inputs:0':[input_user], 
                         'training_model/encoder_inputs_length:0':[len(input_user)]})

INFO:tensorflow:Restoring parameters from ../chkpt/seq2seq_twitter_testing-5501


In [93]:
np.argsort(q_0[3][0])[:3] // vocab_size

array([0, 1, 2], dtype=int64)

In [85]:
q_0[3][0][[3802, 21125, 38448]]

array([-40.74815369, -40.74814987, -40.74814987], dtype=float32)

In [66]:
q_0[2][0][:17323], q_0[2][0][17323:17323*2]#, q_0[2][0][17323*2:]

(array([-33.68927002, -27.15158844, -27.81147194, ..., -16.75342941,
        -25.89280319, -28.28565025], dtype=float32),
 array([-33.6892662 , -27.15158844, -27.81147385, ..., -16.75342941,
        -25.89280319, -28.28565216], dtype=float32))

In [510]:
def _check_maybe(t):
  if isinstance(t, tensor_array_ops.TensorArray):
    raise TypeError(
        "TensorArray state is not supported by BeamSearchDecoder: %s" % t.name)
  if t.shape.ndims is None:
    raise ValueError(
        "Expected tensor (%s) to have known rank, but ndims == None." % t)

In [108]:
previously_finished = _tensor_gather_helper(
      gather_indices=next_beam_ids,
      gather_from=previously_finished,
      batch_size=batch_size,
      range_size=beam_width,
      gather_shape=[-1])
next_finished = math_ops.logical_or(previously_finished,
                                      math_ops.equal(next_word_ids, end_token))

In [520]:
next_finished = math_ops.logical_or(previously_finished,
                                      math_ops.equal(next_word_ids, end_token))

<tf.Tensor 'ToInt32_1:0' shape=(?, 3) dtype=int32>

In [512]:
next_cell_state = nest.map_structure(
  lambda gather_from: _maybe_tensor_gather_helper(
      gather_indices=next_beam_ids,
      gather_from=gather_from,
      batch_size=_batch_size,
      range_size=beam_width,
      gather_shape=[batch_size * beam_width, -1]),
  next_cell_state)

In [526]:
next_cell_state[0].shape

AttributeError: 'LSTMStateTuple' object has no attribute 'shape'

In [522]:
with tf.Session() as sess:
    #sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver()
   
    saver.restore(sess, '../chkpt/seq2seq_twitter_testing-5501')
    q_1 = sess.run([next_cell_state, next_beam_ids],
                     feed_dict={'training_model/encoder_inputs:0':[input_user], 
                         'training_model/encoder_inputs_length:0':[len(input_user)]})
    #print(math_ops.equal(next_word_ids, 1).eval())
    #print(word_indices.eval(), next_beam_ids.eval())
    #j0 = matrix_test_flat.eval()

INFO:tensorflow:Restoring parameters from ../chkpt/seq2seq_twitter_testing-5501


INFO:tensorflow:Restoring parameters from ../chkpt/seq2seq_twitter_testing-5501


In [523]:
q_1[1]

array([[0, 0, 0]])

In [519]:
q_0[1]

(LSTMStateTuple(c=array([[[  1.07746446e+00,   1.39833891e+00,  -7.95096517e-01,
          -9.88991261e-01,   3.54261696e-01,  -1.26826316e-01,
           1.11133492e+00,   1.68521166e-01,  -3.31944609e+00,
          -2.36849338e-01,  -1.34005025e-01,   3.41495097e-01,
          -1.51209676e+00,   4.12390605e-02,   6.42023206e-01,
           6.45930469e-01,  -8.39475572e-01,   4.59602997e-02,
           3.79202276e-01,  -4.25186962e-01,  -4.10487473e-01,
          -1.13491392e+00,   3.00715327e+00,  -1.55043423e-01,
           2.13549709e+00,  -4.15373504e-01,  -6.50357962e-01,
           1.73123503e+00,  -1.28164279e+00,   1.47661555e+00,
           5.90718627e-01,  -4.38636094e-01,  -2.48441070e-01,
           7.02061772e-01,  -1.28343076e-01,  -8.15171957e-01,
           6.60856485e-01,  -1.29501092e+00,  -7.64728427e-01,
          -1.31981468e+00,  -1.03026807e+00,   1.08972065e-01,
          -1.59643865e+00,   1.03445923e+00,  -6.58498406e-01,
           6.47295415e-01,  -1.222254

In [196]:
q_0[1].cell_state[0].c.shape, q_0[3][0].c.shape, q[0][0].c.shape

((1, 3, 256), (1, 3, 256), (1, 3, 256))

In [200]:
q_0[1].cell_state[0].c

array([[[ -3.65474850e-01,   1.12576163e+00,  -3.74980420e-01,
           1.50992310e+00,   5.37272036e-01,   3.91961157e-01,
          -1.52826953e+00,  -7.57203043e-01,  -8.20555866e-01,
           3.28928903e-02,  -9.98938307e-02,   2.73983389e-01,
           1.43370569e+00,  -1.25262678e+00,   8.46861660e-01,
          -8.68561864e-03,   2.23484948e-01,  -7.53591537e-01,
           2.77316332e-01,  -5.83034694e-01,   1.11584201e-01,
          -2.11032346e-01,  -2.25653961e-01,  -5.18377364e-01,
           1.30274057e+00,   4.20663729e-02,  -5.48229694e-01,
           8.91334355e-01,   2.13762307e+00,   1.28638840e+00,
           1.11455381e-01,  -3.87726396e-01,  -4.87494588e-01,
           4.77665514e-01,   1.71412349e+00,  -2.49472666e+00,
          -2.32936785e-01,   1.04379809e+00,  -2.56766081e-01,
          -7.47109413e-01,   1.53968561e+00,   2.33146995e-01,
          -1.78095296e-01,  -4.20007288e-01,  -1.23017883e+00,
          -5.03628552e-01,   9.02279079e-01,  -6.476871

In [186]:
q_0[3][0].c[0][0][:50]

array([-0.4609904 ,  1.48890638,  0.40740454,  1.58216584,  0.33671659,
        0.33087045, -1.09861994, -0.62508821, -0.10278908,  0.19476914,
        0.00553114,  0.84833658,  0.56597167, -0.07370995,  1.34623885,
        0.69166648, -0.03022901, -0.21342668, -0.10207646,  0.31170404,
        0.01683754, -0.75263631,  0.30970672, -0.39155754,  1.78106308,
        0.38375923, -0.40564755,  0.19589956,  2.98818994,  1.05087876,
        0.15760297, -0.20615053, -0.19378714, -0.27645218,  0.49231607,
       -2.08679676, -1.10322309,  1.04165912, -0.4428795 , -0.07466602,
        0.69417888, -0.64663017,  0.23698464,  0.25072286, -0.79507834,
       -1.20115471,  0.17505749,  0.25558031, -1.09977806,  1.36047328], dtype=float32)

In [187]:
q[0][0].c[0][0][:50]

array([-0.07892297,  0.69406658,  1.52863657, -0.48017117,  0.16653439,
       -0.78246737,  1.16757715, -1.43524468,  1.31203496, -0.07768251,
        0.32296625,  0.98345715, -0.54718858, -1.01873028, -1.76884067,
        0.94610697, -0.7862218 ,  1.7737571 , -1.4930712 ,  0.55710387,
        0.77426898,  0.59696943, -0.75783592,  0.22331113,  0.58374703,
        0.60095996, -0.29346567,  0.5867362 ,  0.26750687,  1.13035989,
        0.90487587,  0.64074546, -0.16253649,  0.64425224,  1.35963213,
       -1.51637876,  0.25202215,  0.7358079 , -0.0115959 ,  1.99456429,
       -0.9392224 ,  0.82293564, -1.0955379 , -2.59314847, -0.13732627,
        0.90380156,  0.62487119, -0.39785492, -0.74642199,  0.55457616], dtype=float32)

In [222]:
q

[array([[ 3.5485158 ,  2.73583937,  2.02656889]], dtype=float32),
 array([[6, 4, 9]]),
 array([[0, 0, 0]]),
 array([[6, 4, 9]])]

In [70]:
config = tf.ConfigProto(device_count = {'GPU': 0})
with tf.Session(config=config) as session:
    saver = tf.train.Saver()
   
    saver.restore(session, '../chkpt/seq2seq_twitter_testing-5501')
    
    t0 = session.run([outputs_0, next_state_0, next_inputs_0, finished_step_0], 
                     feed_dict = { 'training_model/encoder_inputs:0':[input_user], 
                         'training_model/encoder_inputs_length:0':[len(input_user)],
                         'training_model/decoder_targets:0':[output_user], 
                         'training_model/decoder_targets_length:0':[len(output_user)]})

    t1 = session.run([outputs_1, next_state_1, next_inputs_1, finished_step_1], 
                     feed_dict = { 'training_model/encoder_inputs:0':[input_user], 
                     'training_model/encoder_inputs_length:0':[len(input_user)],
                     'training_model/decoder_targets:0':[output_user], 
                     'training_model/decoder_targets_length:0':[len(output_user)]})

session.close()

INFO:tensorflow:Restoring parameters from ../chkpt/seq2seq_twitter_testing-5501


INFO:tensorflow:Restoring parameters from ../chkpt/seq2seq_twitter_testing-5501


In [76]:
np.argmax(t0[0].rnn_output[0])

4400

In [85]:
t0[1]

(LSTMStateTuple(c=array([[  1.07746494e+00,   1.39833879e+00,  -7.95096159e-01,
         -9.88991618e-01,   3.54261518e-01,  -1.26826763e-01,
          1.11133444e+00,   1.68521285e-01,  -3.31944585e+00,
         -2.36849368e-01,  -1.34005010e-01,   3.41495335e-01,
         -1.51209688e+00,   4.12387252e-02,   6.42023265e-01,
          6.45930171e-01,  -8.39475930e-01,   4.59600650e-02,
          3.79202396e-01,  -4.25186604e-01,  -4.10487503e-01,
         -1.13491428e+00,   3.00715303e+00,  -1.55043423e-01,
          2.13549781e+00,  -4.15373653e-01,  -6.50358021e-01,
          1.73123527e+00,  -1.28164291e+00,   1.47661567e+00,
          5.90718269e-01,  -4.38636065e-01,  -2.48441324e-01,
          7.02061892e-01,  -1.28343046e-01,  -8.15172017e-01,
          6.60856724e-01,  -1.29501116e+00,  -7.64728546e-01,
         -1.31981480e+00,  -1.03026748e+00,   1.08971879e-01,
         -1.59643888e+00,   1.03445911e+00,  -6.58498645e-01,
          6.47295654e-01,  -1.22225419e-01,   5.9283

In [72]:
t1[0]

BasicDecoderOutput(rnn_output=array([[ -8.97474957,  19.76351929, -11.02628422, ...,  -3.11950421,
         -4.76721382,  -2.02327013]], dtype=float32), sample_id=array([15014]))

In [73]:
inv_map[15014]

'game'

In [52]:
inv_map[4400]

'last'

In [741]:
import beam_search

In [853]:
from importlib import reload
reload(beam_search)

<module 'beam_search' from 'd:\\coding\\seq2seq\\beam_search.py'>

In [866]:
np.array([data_formatting.decodeSent(list(zip(*test[0][0]))[i], inv_map) for i in range(2)])

array([list(['last', 'night', '<EOS>', '<EOS>', '<EOS>', '<EOS>']),
       list(['last', 'night', '<EOS>'])], dtype=object)

In [852]:
vocab_dict['?']

10150

In [884]:
np.array([data_formatting.decodeSent(q[i]['decoder_seq'], inv_map) for i in range(18)])

array([['last', 'today', 'yes', 'show', 'did', 'playing', 'fire', 'over',
        'fire', 'rt', 'soon', '<EOS>'],
       ['last', 'today', 'yes', 'show', 'did', 'playing', 'fire', 'over',
        'fire', 'rt', 'next', '<EOS>'],
       ['last', 'today', 'show', 'show', 'did', 'playing', 'fire', 'over',
        'fire', 'rt', 'soon', '<EOS>'],
       ['last', 'today', 'show', 'show', 'did', 'playing', 'fire', 'over',
        'fire', 'rt', 'next', '<EOS>'],
       ['last', 'today', 'yes', 'show', 'did', 'right', 'fire', 'over',
        'fire', 'rt', 'soon', '<EOS>'],
       ['last', 'today', 'yes', 'show', 'did', 'right', 'fire', 'over',
        'fire', 'rt', 'next', '<EOS>'],
       ['last', 'today', 'show', 'show', 'did', 'right', 'fire', 'over',
        'fire', 'rt', 'soon', '<EOS>'],
       ['last', 'today', 'show', 'show', 'did', 'right', 'fire', 'over',
        'fire', 'rt', 'next', '<EOS>'],
       ['last', 'today', 'yes', 'show', 'being', 'playing', 'fire', 'over',
        'fire', 

In [890]:
beams, new_beams, results = [(1, 0, {'eos': 0, 'dec_inp': [], 'prob': 1, 'prob_ts': 1, 'prob_t': 1})], [], []

In [896]:
x=tf.placeholder(dtype=bool)
y=tf.placeholder(dtype=bool)

In [900]:
q = tf.logical_or(x,y)

In [899]:
sess =tf.Session()

In [905]:
sess.run(q, feed_dict={x:False, y:True})

True

In [893]:
min([1,2])

1

In [885]:
q = beam_search.beamDecode(100, inf_train, input_user, '../chkpt/seq2seq_twitter_testing-5501', 100, vocab_dict['?'], debug=False)

INFO:tensorflow:Restoring parameters from ../chkpt/seq2seq_twitter_testing-5501


INFO:tensorflow:Restoring parameters from ../chkpt/seq2seq_twitter_testing-5501


CURRENT STEP 0
CURRENT STEP 1
CURRENT STEP 2
CURRENT STEP 3
CURRENT STEP 4
CURRENT STEP 5
CURRENT STEP 6
CURRENT STEP 7
CURRENT STEP 8
CURRENT STEP 9
CURRENT STEP 10
CURRENT STEP 11
CURRENT STEP 12
CURRENT STEP 13
CURRENT STEP 14
CURRENT STEP 15
CURRENT STEP 16
CURRENT STEP 17
CURRENT STEP 18
CURRENT STEP 19
CURRENT STEP 20
CURRENT STEP 21
CURRENT STEP 22
CURRENT STEP 23
CURRENT STEP 24
CURRENT STEP 25
CURRENT STEP 26
CURRENT STEP 27
CURRENT STEP 28
CURRENT STEP 29
CURRENT STEP 30
CURRENT STEP 31
CURRENT STEP 32
CURRENT STEP 33
CURRENT STEP 34
CURRENT STEP 35
CURRENT STEP 36
CURRENT STEP 37
CURRENT STEP 38
CURRENT STEP 39
CURRENT STEP 40
CURRENT STEP 41
CURRENT STEP 42
CURRENT STEP 43
CURRENT STEP 44
CURRENT STEP 45
CURRENT STEP 46
CURRENT STEP 47
CURRENT STEP 48
CURRENT STEP 49
CURRENT STEP 50
CURRENT STEP 51
CURRENT STEP 52
CURRENT STEP 53
CURRENT STEP 54
CURRENT STEP 55
CURRENT STEP 56
CURRENT STEP 57
CURRENT STEP 58
CURRENT STEP 59
CURRENT STEP 60
CURRENT STEP 61


KeyboardInterrupt: 

In [873]:
config = tf.ConfigProto(device_count = {'GPU': 0})
 
with tf.Session(config=config) as session:

    saver = tf.train.Saver()

    saver.restore(session, '../chkpt/seq2seq_twitter_testing-5501')

    test = session.run([inf_model_2.decoder_pred_decode
                           ],
           feed_dict = { 'training_model/encoder_inputs:0':[input_user], 
                         'training_model/encoder_inputs_length:0':[len(input_user)]})
session.close()

INFO:tensorflow:Restoring parameters from ../chkpt/seq2seq_twitter_testing-5501


INFO:tensorflow:Restoring parameters from ../chkpt/seq2seq_twitter_testing-5501


In [839]:
data_formatting.decodeSent(test[0][0], inv_map) #Greedy result is exactly decoder input

['last', 'game', 'of', 'the', 'regular', 'season', '<EOS>']

In [875]:
[data_formatting.decodeSent(list(zip(*test[0][0]))[i], inv_map) for i in range(100)]

[['last', 'night', '<EOS>', '<EOS>', '<EOS>', '<EOS>'],
 ['last', 'night', '<EOS>'],
 ['halloween', 'is', 'playing', 'this', 'thread', '<EOS>', '<EOS>'],
 ['halloween', 'is', 'playing', 'this', 'thread', '<EOS>'],
 ['last', 'game', 'of', 'the', 'box', 'and', 'today', '<EOS>'],
 ['last', 'night', 'with', 'exactly', '<EOS>', '<EOS>', '<EOS>', '<EOS>'],
 ['last', 'night', 'with', 'exactly', '<EOS>'],
 ['new', 'york', 'though', '<EOS>', '<EOS>'],
 ['new', 'york', 'though', '<EOS>'],
 ['my', 'apartment', 'exactly', '<EOS>'],
 ['hey', 'game', 'though', '<EOS>', '<EOS>'],
 ['new', 'york', 'though', '<EOS>', '<EOS>', '<EOS>'],
 ['last', 'game', 'of', 'the', 'ny', '<EOS>', '<EOS>'],
 ['hey', 'folk', 'send', 'me', '?', '<EOS>', '<EOS>'],
 ['hey', 'game', 'though', '<EOS>'],
 ['halloween', 'and', '?', '<EOS>', '<EOS>'],
 ['last', 'game', 'of', 'the', 'ny', '<EOS>'],
 ['hey', 'folk', 'see', 'not', 'it', '<EOS>', '<EOS>', '<EOS>', '<EOS>'],
 ['hey', 'folk', 'send', 'me', '?', '<EOS>'],
 ['hey', 'bo

In [425]:
[inv_map[i] for i in test[0][0]]

['i',
 'will',
 'be',
 'a',
 'copy',
 'of',
 'the',
 'same',
 'of',
 'the',
 'same',
 '?',
 '<EOS>']

In [449]:
from collections import OrderedDict

def executeStep(session, encoder_input, encoder_input_length, decoder_target, decoder_target_length):

    test = session.run([inf_train.decoder_pred_train_prob],
           feed_dict = { 'training_model/encoder_inputs:0':encoder_input, 
                         'training_model/encoder_inputs_length:0':encoder_input_length,
                         'training_model/decoder_targets:0':decoder_target, 
                         'training_model/decoder_targets_length:0':decoder_target_length})

    return test[0]

In [539]:
np.array(2.3, dtype=np.float64)

array(2.3)

In [482]:
input_user = [i for i in train_data[0][31] if i!=0]
beam_width = 2

In [483]:
seq_len = 15

In [644]:
beam = [0, input_user, len(input_user), [], 0, 0] #'eos, encoder_seq, encoder_seq_length, decoder_seq, decoder_seq_length, prob

In [787]:
data_formatting.decodeSent(list(zip(*test[0][0]))[0], inv_map)

['i',
 'will',
 'be',
 'a',
 'copy',
 'of',
 'this',
 '?',
 'i',
 'will',
 'have',
 'a',
 'copy',
 'of',
 'the',
 'same',
 '?',
 '<EOS>',
 '<EOS>']

In [864]:
#gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=1)
config = tf.ConfigProto(device_count = {'GPU': 0})
with tf.Session(config=config) as session:
    
    saver = tf.train.Saver()
   
    #saver.restore(session, 'd:\coding\seq2seq_2\seq2seq\chkpt\seq2seq_twitter_queue-1')
    saver.restore(session, '../chkpt/seq2seq_enron_testing-501')
    
    #train_data_encoder = data_formatting.encodeSent(['hi', 'enough', 'got', 'it', 'yet', '<EOS>'], vocab_dict)

    train_data_encoder = input_user
    
    train_data_decoder = [4459]
    #inf = session.run([inf_model_1.decoder_pred_decode
    #                       ],
    #       feed_dict = { 'training_model/encoder_inputs:0':[train_data_encoder], 
    #                     'training_model/encoder_inputs_length:0':[len(train_data_encoder)]})

    test = session.run([inf_model_2.decoder_pred_decode
                           ],
           feed_dict = { 'training_model/encoder_inputs:0':[train_data_encoder], 
                         'training_model/encoder_inputs_length:0':[len(train_data_encoder)]})

    #inf_initialize = session.run([inf_model_2.inference_decoder.initialize()
    #                       ],
    #       feed_dict = {'training_model/encoder_inputs:0':[train_data_encoder], 
    #                     'training_model/encoder_inputs_length:0':[len(train_data_encoder)]})

   # inf_step_0 = session.run([inf_model_2.inference_decoder.step(0, first_inputs, initial_state)],
   #        feed_dict = {'training_model/encoder_inputs:0':[train_data_encoder], 
   #                      'training_model/encoder_inputs_length:0':[len(train_data_encoder)]})
    
   
    #inf_step_1 = session.run([inf_model_2.inference_decoder.step(1, input_beam, input_state_beam)],
    #       feed_dict = {'training_model/encoder_inputs:0':[train_data_encoder], 
    #                     'training_model/encoder_inputs_length:0':[len(train_data_encoder)]})
    
    
    
   #inf_out = session.run([inf_model.decoder_pred_decode, inf_model.decoder_pred_decode_prob], feed_dict_inf)
   # inf_out = session.run([#inf_model.decoder_outputs_decode_beam, 
      #                     inf_model_1.decoder_outputs_decode, inf_model_2.decoder_outputs_decode
     #                      ],
     #       feed_dict = {'training_model/encoder_inputs:0':[train_data_encoder], 
      #                   'training_model/encoder_inputs_length:0':[len(train_data_encoder)],
     ##                   'training_model_1/encoder_inputs:0':[train_data_encoder], 
    #                     'training_model_1/encoder_inputs_length:0':[len(train_data_encoder)]})
    
    #encoder_input = inf_out[0]
    #decoder_target = train_data_decoder
    ##decoder_inference = inf_out[1]

  #  for idx, e_in in enumerate(encoder_input):
                
   #     print ('###%d###' % idx)
        
    #    print ('INPUT:', data_formatting.decodeSent(e_in, inv_map))
     #   print ('OUTPUT:', data_formatting.decodeSent(decoder_target, inv_map))        
      #  try:
       #     beam_inf = list(zip(*decoder_inference[idx]))
        #    for idx_inf, dt_inf in enumerate(beam_inf):

         #       print ('INFERENCE:', data_formatting.decodeSent(dt_inf, inv_map))
          #      if idx_inf>20: break                
        #except:
         #   print ('INFERENCE:', data_formatting.decodeSent(decoder_inference[idx], inv_map))

INFO:tensorflow:Restoring parameters from ../chkpt/seq2seq_enron_testing-501


INFO:tensorflow:Restoring parameters from ../chkpt/seq2seq_enron_testing-501


NotFoundError: Key training_model/decoder/multi_rnn_cell/cell_0/layer_norm_basic_lstm_cell/input/beta not found in checkpoint
	 [[Node: save/RestoreV2_2 = RestoreV2[dtypes=[DT_FLOAT], _device="/job:localhost/replica:0/task:0/cpu:0"](_arg_save/Const_0_0, save/RestoreV2_2/tensor_names, save/RestoreV2_2/shape_and_slices)]]

Caused by op 'save/RestoreV2_2', defined at:
  File "c:\users\euix\anaconda3\envs\tensorflow-gpu\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\users\euix\anaconda3\envs\tensorflow-gpu\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "c:\users\euix\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "c:\users\euix\anaconda3\envs\tensorflow-gpu\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "c:\users\euix\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel\kernelapp.py", 

In [736]:
beams = [{'eos':0, 'encoder_seq':input_user, 'encoder_seq_length': len(input_user), 
       'decoder_seq':[], 'decoder_seq_length':0, 'total_log_prob':0} for i in range(beam_width)]

config = tf.ConfigProto(device_count = {'GPU': 0})

session = tf.Session(config=config)

saver = tf.train.Saver()

saver.restore(session, '../chkpt/seq2seq_enron_testing-501')

for pos in range(100):
#while beams_remaining!=0:    
    print ('CURRENT POS', pos)

    #if pos>1: break
    encoder_input = [beams[i]['encoder_seq'] for i in range(len(beams)) if beams[i]['eos']!=1]
    encoder_input_length = [beams[i]['encoder_seq_length'] for i in range(len(beams)) if beams[i]['eos']!=1]
    decoder_targets = [beams[i]['decoder_seq'] for i in range(len(beams)) if beams[i]['eos']!=1]
    decoder_targets_length = [beams[i]['decoder_seq_length'] for i in range(len(beams)) if beams[i]['eos']!=1]
    
    beams_remaining = len(encoder_input)
    
    if beams_remaining == 0 :
        print ('Finished Beam search')
        break
    
    print ('seq_lengths', encoder_input_length, decoder_targets_length)
    logits = executeStep(session, encoder_input, encoder_input_length, decoder_targets, decoder_targets_length)

    beam_results = np.concatenate([argMaxTopN(logits[i][-1], beam_width) for i in range(beams_remaining)])
    print (beam_results)
    
    beam_results_vocab = list(np.concatenate([i for idx, i in enumerate(beam_results) if idx%2==0]))
    beam_results_probs = list(np.concatenate([i for idx, i in enumerate(beam_results) if idx%2==1]))
                                        
    beam_results_vocab = list(OrderedDict.fromkeys(beam_results_vocab))
    
    beam_results_probs = [beam_results_probs[beam_results_vocab.index(i)] for i in beam_results_vocab]
    
    #beam_results_probs = list(OrderedDict.fromkeys(np.concatenate([i for idx, i in enumerate(beam_results) if idx%2==1])))
    print ('beam_results_vocab ', beam_results_vocab)

    print ('beam_results_probs ', beam_results_probs)
    top_n_index = np.argsort(beam_results_probs)[::-1][:beam_width]
    print ('top_n_index ', top_n_index)
    
    temp_beam_list = []
    
    for idx_beam, beam in enumerate(beams):
        temp_beam = copy.deepcopy(beam)
        
        for idx, arg in enumerate(top_n_index[:beam_width]):
        
            print ('IDX, ARG', idx, arg, idx_beam)
            print ('CURRENT BEAM', temp_beam)
      
            if temp_beam['eos']==1:
                continue
            
            temp_beam['total_log_prob']+=beam_results_probs[arg]
            
            temp_beam['decoder_seq'].append(beam_results_vocab[arg])

            if beam_results_vocab[arg] == 1: 
                temp_beam['eos'] = 1
        
            temp_beam['decoder_seq_length']+=1
            
            if True not in [temp_beam['decoder_seq']==current_beam['decoder_seq'] for current_beam in temp_beam_list]:
            
                temp_beam_list.append(temp_beam)
            
            temp_beam = copy.deepcopy(beam)
            
            print ('TEMP BEAM', temp_beam)
    beams_all = sorted(temp_beam_list, key=lambda x: x['total_log_prob'], reverse=True)
    #print (beams_all)
    beams = beams_all[:beam_width]
    #print (beams)

session.close()

INFO:tensorflow:Restoring parameters from ../chkpt/seq2seq_enron_testing-501


INFO:tensorflow:Restoring parameters from ../chkpt/seq2seq_enron_testing-501


CURRENT POS 0
seq_lengths [31, 31] [0, 0]
[[  9.99000000e+02   4.55900000e+03]
 [ -2.74616718e+00  -2.40978003e+00]
 [  9.99000000e+02   4.55900000e+03]
 [ -2.74616718e+00  -2.40978003e+00]]
beam_results_vocab  [999.0, 4559.0]
beam_results_probs  [-2.7461671829223633, -2.4097800254821777]
top_n_index  [1 0]
IDX, ARG 0 1 0
CURRENT BEAM {'encoder_seq': [1328, 340, 5846, 4812, 4915, 6671, 2848, 4234, 1357, 1380, 6487, 2848, 4234, 6487, 2158, 4323, 4491, 4643, 2093, 6399, 6999, 4742, 7228, 4749, 5519, 4812, 3631, 6727, 783, 2093, 1], 'encoder_seq_length': 31, 'decoder_seq_length': 0, 'total_log_prob': 0, 'decoder_seq': [], 'eos': 0}
TEMP BEAM {'encoder_seq': [1328, 340, 5846, 4812, 4915, 6671, 2848, 4234, 1357, 1380, 6487, 2848, 4234, 6487, 2158, 4323, 4491, 4643, 2093, 6399, 6999, 4742, 7228, 4749, 5519, 4812, 3631, 6727, 783, 2093, 1], 'encoder_seq_length': 31, 'decoder_seq_length': 0, 'total_log_prob': 0, 'decoder_seq': [], 'eos': 0}
IDX, ARG 1 0 0
CURRENT BEAM {'encoder_seq': [1328, 34

[[  6.43400000e+03   4.59800000e+03]
 [ -2.06361961e+00  -8.27529848e-01]
 [  6.43400000e+03   4.59800000e+03]
 [ -2.50332165e+00  -9.75676477e-01]]
beam_results_vocab  [6434.0, 4598.0]
beam_results_probs  [-2.0636196136474609, -0.82752984762191772]
top_n_index  [1 0]
IDX, ARG 0 1 0
CURRENT BEAM {'encoder_seq': [1328, 340, 5846, 4812, 4915, 6671, 2848, 4234, 1357, 1380, 6487, 2848, 4234, 6487, 2158, 4323, 4491, 4643, 2093, 6399, 6999, 4742, 7228, 4749, 5519, 4812, 3631, 6727, 783, 2093, 1], 'encoder_seq_length': 31, 'decoder_seq_length': 4, 'total_log_prob': -5.6029229573905468, 'decoder_seq': [4559.0, 6703.0, 232.0, 1634.0], 'eos': 0}
TEMP BEAM {'encoder_seq': [1328, 340, 5846, 4812, 4915, 6671, 2848, 4234, 1357, 1380, 6487, 2848, 4234, 6487, 2158, 4323, 4491, 4643, 2093, 6399, 6999, 4742, 7228, 4749, 5519, 4812, 3631, 6727, 783, 2093, 1], 'encoder_seq_length': 31, 'decoder_seq_length': 4, 'total_log_prob': -5.6029229573905468, 'decoder_seq': [4559.0, 6703.0, 232.0, 1634.0], 'eos': 0}

[[  2.09300000e+03   1.00000000e+00]
 [ -2.06707144e+00  -1.59415483e+00]
 [  2.09300000e+03   1.00000000e+00]
 [ -2.05004454e+00  -1.75532830e+00]]
beam_results_vocab  [2093.0, 1.0]
beam_results_probs  [-2.0670714378356934, -1.5941548347473145]
top_n_index  [1 0]
IDX, ARG 0 1 0
CURRENT BEAM {'encoder_seq': [1328, 340, 5846, 4812, 4915, 6671, 2848, 4234, 1357, 1380, 6487, 2848, 4234, 6487, 2158, 4323, 4491, 4643, 2093, 6399, 6999, 4742, 7228, 4749, 5519, 4812, 3631, 6727, 783, 2093, 1], 'encoder_seq_length': 31, 'decoder_seq_length': 9, 'total_log_prob': -8.6640370301902294, 'decoder_seq': [4559.0, 6703.0, 232.0, 1634.0, 4598.0, 2552.0, 3929.0, 2686.0, 6567.0], 'eos': 0}
TEMP BEAM {'encoder_seq': [1328, 340, 5846, 4812, 4915, 6671, 2848, 4234, 1357, 1380, 6487, 2848, 4234, 6487, 2158, 4323, 4491, 4643, 2093, 6399, 6999, 4742, 7228, 4749, 5519, 4812, 3631, 6727, 783, 2093, 1], 'encoder_seq_length': 31, 'decoder_seq_length': 9, 'total_log_prob': -8.6640370301902294, 'decoder_seq': [4559.

In [21]:
INFO:tensorflow:Restoring parameters from ../chkpt/seq2seq_enron_testing-2001
###0###
INPUT: ['the', 'need', 'to', 'have', 'rick', 'involved', 'thanks', '<EOS>']
OUTPUT: ['in', 'to', 'the', 'meeting', 'the', 'need', 'to', 'have', 'rick', 'involved', 'thanks', '<EOS>']
INFERENCE: ['in', 'to', 'the', 'meeting', 'the', 'need', 'to', 'have', 'rick', 'involved', 'thanks', '<EOS>']
INFERENCE: ['in', 'to', 'the', 'list', '<EOS>']
INFERENCE: ['in', 'to', 'the', 'meeting', 'the', 'need', 'to', 'have', 'rick', 'involved', '<EOS>', '<EOS>']
INFERENCE: ['in', 'to', 'the', 'meeting', 'the', 'need', 'to', 'have', 'rick', 'involved', '<EOS>']
INFERENCE: ['in', 'to', 'the', 'meeting', 'the', 'need', 'to', 'need', 'to', 'have', 'you', 'involved', 'thanks', '<EOS>', '<EOS>', '<EOS>']
INFERENCE: ['in', 'to', 'the', 'time', 'book', 'the', 'information', '<EOS>']
INFERENCE: ['in', 'to', 'the', 'meeting', 'the', 'need', 'to', 'need', 'to', 'have', 'you', 'involved', 'thanks', '<EOS>']
INFERENCE: ['in', 'to', 'the', 'meeting', 'the', 'need', 'to', 'have', 'attend', '<EOS>']
INFERENCE: ['in', 'to', 'the', 'call', 'change', 'i', 'need', 'to', 'have', 'the', 'meeting', 'folks', 's', 'a', 'daily', 'daily', '<EOS>']
INFERENCE: ['in', 'to', 'the', 'call', 'change', 'i', 'need', 'to', 'have', 'the', 'meeting', 'jeff', 'need', 'to', 'have', 'there', 'involved', '<EOS>']

array([list(['voice', 'mail', 'message', '<EOS>']),
       list(['twice', 'but', 'still', 'haven', 't', 'received', 'it', '<EOS>']),
       list([4571, 3723, 3576, 1]),
       list([4267, 4887, 540, 2019, 1505, 456, 6344, 1]), 1], dtype=object)

In [43]:
[n.name for n in tf.get_default_graph().as_graph_def().node]

['training_model/encoder_inputs',
 'training_model/encoder_inputs_length',
 'training_model/Shape',
 'training_model/strided_slice/stack',
 'training_model/strided_slice/stack_1',
 'training_model/strided_slice/stack_2',
 'training_model/strided_slice',
 'training_model/embedding_matrix/Initializer/random_uniform/shape',
 'training_model/embedding_matrix/Initializer/random_uniform/min',
 'training_model/embedding_matrix/Initializer/random_uniform/max',
 'training_model/embedding_matrix/Initializer/random_uniform/RandomUniform',
 'training_model/embedding_matrix/Initializer/random_uniform/sub',
 'training_model/embedding_matrix/Initializer/random_uniform/mul',
 'training_model/embedding_matrix/Initializer/random_uniform',
 'training_model/embedding_matrix',
 'training_model/embedding_matrix/Assign',
 'training_model/embedding_matrix/read',
 'training_model/embedding_lookup',
 'training_model/DropoutWrapperInit/Const',
 'training_model/DropoutWrapperInit/Const_1',
 'training_model/Dropou

In [19]:
output_node_names = [n.name for n in tf.get_default_graph().as_graph_def().node 
                         if 'decoder_pred' in n.name or 'encoder_input' in n.name]

In [20]:
output_node_names

['training_model/encoder_inputs',
 'training_model/encoder_inputs_length',
 'training_model/decoder_pred_decode']

In [49]:
[var for var in q if 'pred' in var.name]

[<tf.Operation 'training_model/decoder/while/BasicDecoderStep/cond/pred_id' type=Identity>,
 <tf.Operation 'training_model/decoder_pred_decode/dimension' type=Const>,
 <tf.Operation 'training_model/decoder_pred_decode' type=ArgMax>]

In [12]:
with tf.Session() as session:
    
    saver = tf.train.Saver()
   
    #saver.restore(session, 'd:\coding\seq2seq_2\seq2seq\chkpt\seq2seq_twitter_queue-1')
    saver.restore(session, '../chkpt/seq2seq_enron_testing-2001')
    q = session.graph.get_operations()
    output_graph_def = tf.graph_util.convert_variables_to_constants(
        session, # The session is used to retrieve the weights
        tf.get_default_graph().as_graph_def(), # The graph_def is used to retrieve the nodes 
        ['training_model/encoder_inputs',
 'training_model/encoder_inputs_length',
 'training_model/decoder_pred_decode'] # The output node names are used to select the usefull nodes
    ) 
    

     #Finally we serialize and dump the output graph to the filesystem
    with tf.gfile.GFile('frozen_model.pb', "wb") as f:
        f.write(output_graph_def.SerializeToString())
    print("%d ops in the final graph." % len(output_graph_def.node))

INFO:tensorflow:Restoring parameters from ../chkpt/seq2seq_enron_testing-2001
INFO:tensorflow:Froze 15 variables.
Converted 15 variables to const ops.
1158 ops in the final graph.


In [2]:
def load_graph(frozen_graph_filename):
    # We load the protobuf file from the disk and parse it to retrieve the 
    # unserialized graph_def
    with tf.gfile.GFile(frozen_graph_filename, "rb") as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())

    # Then, we import the graph_def into a new Graph and returns it 
    with tf.Graph().as_default() as graph:
        # The name var will prefix every op/nodes in your graph
        # Since we load everything in a new graph, this is not needed
        tf.import_graph_def(graph_def)
    return graph

In [3]:
from tensorflow.contrib.seq2seq.python.ops import beam_search_ops

In [5]:
# We use our "load_graph" function
graph = load_graph('..//seq2seq_twitter_n_cells_512_limit_decode_steps_None_vocab_size_38752_embedding_size_2048_num_layers_2_beam_width_10.pb')

vocab_dict = pickle.load(open('../processed_data/word_dict_v02_twitter_py35_seq_length_3_25_sample_1764604_questions.pkl', 'rb'))

inv_map = data_formatting.createInvMap(vocab_dict)

In [6]:
# We can verify that we can access the list of operations in the graph
for op in graph.get_operations():
    print(op.name)

import/training_model/encoder_inputs
import/training_model/encoder_inputs_length
import/training_model/Shape
import/training_model/strided_slice/stack
import/training_model/strided_slice/stack_1
import/training_model/strided_slice/stack_2
import/training_model/strided_slice
import/training_model/embedding_matrix
import/training_model/embedding_matrix/read
import/training_model/embedding_lookup
import/training_model/Rank
import/training_model/range/start
import/training_model/range/delta
import/training_model/range
import/training_model/concat/values_0
import/training_model/concat/axis
import/training_model/concat
import/training_model/transpose
import/training_model/sequence_length
import/training_model/rnn/Shape
import/training_model/rnn/strided_slice/stack
import/training_model/rnn/strided_slice/stack_1
import/training_model/rnn/strided_slice/stack_2
import/training_model/rnn/strided_slice
import/training_model/rnn/MultiRNNCellZeroState/DropoutWrapperZeroState/LSTMCellZeroState/Expan

import/training_model/decoder/while/BeamSearchDecoderStep/Reshape_35
import/training_model/decoder/while/BeamSearchDecoderStep/Reshape_36/shape/1
import/training_model/decoder/while/BeamSearchDecoderStep/Reshape_36/shape
import/training_model/decoder/while/BeamSearchDecoderStep/Reshape_36
import/training_model/decoder/while/BeamSearchDecoderStep/Gather_6
import/training_model/decoder/while/BeamSearchDecoderStep/Shape_24
import/training_model/decoder/while/BeamSearchDecoderStep/strided_slice_21/stack
import/training_model/decoder/while/BeamSearchDecoderStep/strided_slice_21/stack_1
import/training_model/decoder/while/BeamSearchDecoderStep/strided_slice_21/stack_2
import/training_model/decoder/while/BeamSearchDecoderStep/strided_slice_21
import/training_model/decoder/while/BeamSearchDecoderStep/Reshape_37
import/training_model/decoder/while/BeamSearchDecoderStep/mul_22/y
import/training_model/decoder/while/BeamSearchDecoderStep/mul_22
import/training_model/decoder/while/BeamSearchDecoder

In [6]:
# We access the input and output nodes 
x = graph.get_tensor_by_name('prefix/training_model/encoder_inputs:0')
x_len = graph.get_tensor_by_name('prefix/training_model/encoder_inputs_length:0')

y = graph.get_tensor_by_name('prefix/training_model/decoder_pred_decode:0')

In [7]:
df_all = pd.read_pickle('../processed_data/processed_data_v02_twitter_py35_seq_length_3_25_sample_1764604_questions.pkl')

In [8]:
df.head()

NameError: name 'df' is not defined

In [13]:
# We launch a Session
with tf.Session(graph=graph) as sess:
    
    #train_data_encoder = [i for i in train_data[0][30] if i!=0]
    train_data_encoder = data_formatting.encodeSent('who is hilary clinton ? <EOS>', vocab_dict)
    #train_data_decoder = train_data[2][30]
    train_data_decoder = None
    inf_out = sess.run(y, feed_dict={x:[train_data_encoder], x_len:[len(train_data_encoder)]})
    
    encoder_input = [train_data_encoder]
    decoder_target = train_data_decoder
    decoder_inference = [inf_out[0]]
    
    for idx, e_in in enumerate(encoder_input):
                
        print ('###%d###' % idx)
        
        print ('INPUT:', data_formatting.decodeSent(e_in, inv_map))
        
        if train_data_decoder is not None:
            print ('OUTPUT:', data_formatting.decodeSent(decoder_target, inv_map))        
        
        try:
            beam_inf = list(zip(*decoder_inference[idx]))
            for idx_inf, dt_inf in enumerate(beam_inf):

                print ('INFERENCE:', data_formatting.decodeSent(dt_inf, inv_map))
                #if idx_inf>20: break                
        except:
            print ('INFERENCE:', data_formatting.decodeSent(decoder_inference[idx], inv_map))

###0###
INPUT: ['who', 'is', 'hilary', 'clinton', '?', '<EOS>']
INFERENCE: ['she', 's', 'a', 'liar', '<EOS>', '<EOS>']
INFERENCE: ['she', 's', 'the', 'best', '<EOS>', '<EOS>']
INFERENCE: ['i', 'think', 'she', 's', 'the', 'only', 'one', '<EOS>', '<EOS>']
INFERENCE: ['i', 'think', 'she', 's', 'the', 'only', 'one', 'who', 'doesn', 't', 'care', 'about', 'her', '<EOS>']
INFERENCE: ['i', 'think', 'she', 's', 'the', 'only', 'one', 'who', 'doesn', 't', 'know', 'what', 'she', 'said', '<EOS>']
INFERENCE: ['she', 's', 'a', 'liar', '<EOS>']
INFERENCE: ['i', 'think', 'she', 's', 'the', 'only', 'one', 'who', 'doesn', 't', 'know', 'what', 'she', 'want', '<EOS>', '<EOS>']
INFERENCE: ['she', 's', 'the', 'best', '<EOS>']
INFERENCE: ['i', 'think', 'she', 's', 'the', 'only', 'one', 'who', 'doesn', 't', 'know', 'what', 'she', 's', 'saying', '<EOS>']
INFERENCE: ['i', 'think', 'she', 's', 'the', 'only', 'one', '<EOS>']
